## Customizing State
- So in all previous Examples we only one thing in the State `messages`
- Let's add some more fields like `name` && `birthday`.
- Which the chatbot use its search tool to find specific information and forwarding them to human to review it.

In [13]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import add_messages

class State(TypedDict):
    messages : Annotated[list, add_messages]
    name:str
    birthday:str


In [ ]:
from dotenv import load_dotenv

from langchain_core.tools import InjectedToolCallId, tool
from langchain_core.messages import ToolMessage

from langgraph.types import Command, interrupt

load_dotenv()

@tool
# Note that because we are generating a ToolMessage for a state update, we
# generally require the ID of the corresponding tool call. We can use
# LangChain's InjectedToolCallId to signal that this argument should not
# be revealed to the model in the tool's schema.
def human_assistance(
    name : str, birthday: str, tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    """Request assistance from a human."""
    human_response = interrupt(
        {
            "question": "Is the is correct?",
            "name" : name,
            "birthday" : birthday
        }
    )

    # If information is correct, update the state as - in.
    if human_response.get("correct","").lower().startwith('y'):
        verified_name = name
        verified_birthday = birthday
        response = "Correct"
    # Otherwise, revieve the information from the human response
    else:
        verified_name = human_response.get("name",name)
        verified_birthday =  human_response.get("birthday",birthday)
        response = f"Made a correction: {human_response}"

    # At this time we explicitly update the state with a Toolmessage inside the tool.

    state_update = {
        "name":verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response,tool_call_id = tool_call_id)]
    }

    # We return a Command object in the tool to update our state.
    return Command(update= state_update)

In [15]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

tool = TavilySearchResults(max_results = 2)
tools = [tool, human_assistance]
llm = ChatOpenAI(model = "gpt-4o-mini", temperature= 0.3)
llm_with_tools = llm.bind_tools(tools)

def chatbot(state:State):
    message = llm_with_tools.invoke(state["messages"])
    
    assert len(message.tool_call) <= 1

    return {"messages": [message]}

graph_builder = StateGraph(State)

# Add Some nodes
graph_builder.add_node("chatbot",chatbot)

tool_node = ToolNode(tools = tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition
)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer = memory)

In [16]:
user_input = {
    "Can you look up when LangGraph was released? "
    "When you have the answer, use the human_assistance tool for review."
}

config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages" : [{"role" : "user", "content" : user_input}]},
    config,
    stream_mode="values"
)

for event in events:
    print(event)
    if "messages" in event:
        event["messages"][-1].pretty_print()

{'messages': [HumanMessage(content=['Can you look up when LangGraph was released? When you have the answer, use the human_assistance tool for review.'], additional_kwargs={}, response_metadata={}, id='b0a99594-a715-4114-b649-fe38121b5daf')]}
================================ Human Message =================================

['Can you look up when LangGraph was released? When you have the answer, use the human_assistance tool for review.']


BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'messages[0].content[0]': expected an object, but got a string instead.", 'type': 'invalid_request_error', 'param': 'messages[0].content[0]', 'code': 'invalid_type'}}